python version,

compare with r version, to make sure everything is the same 

In [1]:
import numpy as np
import pandas as pd
from scipy import linalg
from scipy.spatial import distance_matrix

In [2]:
def L_num(n):
    t = np.zeros([np.int(n*(n-1)/2),2])
    count = -1
    for i in range(n-1):
        for j in range((i+1),n,1):
            count = count + 1
            t[count,0] = i; t[count,1] = j
    return(t)

def elk(n,p):
    count = -1
    el1 = el2 = np.zeros([n,np.int(n*(n-1)/2)])
    el1 = np.r_[np.eye(1),np.zeros([n-1,1])]
    for i in range(2,n,1):
        el1 = np.c_[el1,np.r_[np.eye(i),np.zeros([n-i,i])]]
    
    count = -1
    for i in range(1,n,1):
        temp = np.zeros([n,i])
        temp[i,:] = 1
        el2[:,(count+1):(count+i+1)] = temp
        count = count+i
        
    count = -1
    ek1 = ek2 = np.zeros([p,np.int(p*(p-1)/2)])
    ek1 = np.r_[np.eye(1),np.zeros([p-1,1])]
    for i in range(2,p,1):
        ek1 = np.c_[ek1,np.r_[np.eye(i),np.zeros([p-i,i])]]

    count = -1
    for i in range(1,p,1):
        temp = np.zeros([p,i])
        temp[i,:] = 1
        ek2[:,(count+1):(count+i+1)] = temp
        count = count+i
    return(el1,el2,ek1,ek2)

def pdist(xs):
    """Pairwise Euclidean distacne."""
    return np.sqrt(np.sum((xs[:, None] - xs[None, :])**2, -1))

def dist_weight(X,phi):
    dist_X = np.tril(pdist(X))[1:,:-1]
    dist_X = [dist_X[j,i] for  i in range(dist_X.shape[0]) for j in range(i,dist_X.shape[0],1)]
    res = np.exp(- phi * np.array(dist_X)**2)
    return(res)

def prox(v,sigma):
    return(np.max([0,(1-sigma/np.sum(v**2))]) * v)
def tri2vec(i,j,n):
    return(n*(i-1) - i*(i-1)/2 + j -i)

In [3]:
def knn_weights(w,k,n): # w input array
    i = 1
    neighbors = [tri2vec(i,j,n) for j in range((i+1),(n+1))]
    temp1 = w[[int(i)-1 for i in neighbors]]
    temp2 = list(-np.sort(-temp1))
    keep = np.array(neighbors)[[list(temp1).index(temp2[i]) for i in range(len(temp2))][0:k]]
    for i in range(2,n,1):
        group_A =  [tri2vec(i,j,n) for j in range((i+1),(n+1))]
        group_B =  [tri2vec(j,i,n) for j in range(1,i)]
        neighbors = group_A
        neighbors.extend(group_B)
        temp1 = w[[int(i)-1 for i in neighbors]]
        temp2 = list(-np.sort(-temp1))
        knn = np.array(neighbors)[[list(temp1).index(temp2[i]) for i in range(len(temp2))][0:k]]
        keep = list(set(knn).union(set(keep)))
    i = n
    neighbors = [tri2vec(j,i,n) for j in range(1,i)]  
    temp1 = w[[int(i)-1 for i in neighbors]]
    temp2 = list(-np.sort(-temp1))
    knn = np.array(neighbors)[[list(temp1).index(temp2[i]) for i in range(len(temp2))][0:k]]
    keep = list(set(knn).union(set(keep)))
    w[np.array(list(set(list(range(1,len(w)+1))) - set(keep)))-1] = 0
    return(w)

In [4]:
def update_A(X, nu1, nu2, lambda_1, lambda_2, v, z,n,p,el1,el2,ek1,ek2):
    En = np.diag(list(range(n))) + np.diag(list(range(n-1,-1,-1))) - np.ones([n,n]) + np.eye(n) 
    Ep = np.diag(list(range(p))) + np.diag(list(range(p-1,-1,-1))) - np.ones([p,p]) + np.eye(p)
    
    M = np.eye(n) + nu1 * En
    N = nu2 * Ep
    
    lv = lambda_1+ nu1 * v
    lz = lambda_2 + nu2 * z
    C2 = 0 -np.dot((el2-el1),lv.T)
    C3 = np.dot(lz,(ek1-ek2).T)
    C = X +  C2 + C3  
   
    A = linalg.solve_sylvester(M, N.T, C)

    return(A)

In [5]:
def update_vz(lambda_1, lambda_2, gamma_1, gamma_2, nu1, nu2, w_l, u_k, al1, al2, ak1, ak2):
    
    sigma_1 = gamma_1 * w_l/nu1
    vtemp = al1 - al2 - 1/nu1 * lambda_1
    
    temp1 = np.where((1 - sigma_1/np.sum(vtemp**2,axis=0) < 0),0,1 - sigma_1/np.sum(vtemp**2,axis=0))
    temp2 = np.repeat(temp1,vtemp.shape[0]).reshape(vtemp.shape[1],vtemp.shape[0]).T * vtemp 
    
    v = temp2
    
    sigma_2 = gamma_2 * u_k/nu2
    ztemp = ak1 - ak2 - 1/nu2 * lambda_2
    
    temp3 = np.where((1 - sigma_2/np.sum(ztemp**2,axis=0) < 0), 0 ,1 - sigma_2/np.sum(ztemp**2,axis=0))
    temp4 = np.repeat(temp3,ztemp.shape[0]).reshape(ztemp.shape[1],ztemp.shape[0]).T * ztemp 
  
    z = temp4
    return(v,z)

In [6]:
def update_lambda(lambda_1,lambda_2,v, z, nu1,nu2, al1, al2, ak1, ak2):
    lambda_1 = lambda_1 + nu1 * (v - al1 + al2)
    lambda_2 = lambda_2 + nu2 * (z - ak1 + ak2)
    return(lambda_1,lambda_2)

In [9]:
X = np.array([[1,5,9],[2,6,10],[3,7,11],[4,8,12]])
X

array([[ 1,  5,  9],
       [ 2,  6, 10],
       [ 3,  7, 11],
       [ 4,  8, 12]])

In [10]:
n=X.shape[0]; p=X.shape[1]

tol = 2

eplison_p=L_num(p)
eplison_n=L_num(n)

elks = elk(n,p)
el1,el2,ek1,ek2 = elks[0],elks[1],elks[2],elks[3]

w = np.array(dist_weight(X / np.sqrt(p),phi = 0.5))
w_l = knn_weights(w,1,n)
u = np.array(dist_weight(X.T / np.sqrt(n),phi = 0.5))
u_k = knn_weights(u,1,p)

A = np.zeros([X.shape[0],X.shape[1]])
v= np.zeros([p,eplison_n.shape[0]])
z = np.zeros([n,eplison_p.shape[0]]) 
lambda_1 = np.zeros([p,eplison_n.shape[0]]) 
lambda_2 = np.zeros([n,eplison_p.shape[0]])

In [16]:
nu1 = nu2 = 1
gamma_1 = 6
gamma_2 = 0

In [13]:
A = update_A(X, nu1, nu2, lambda_1, lambda_2, v, z,n,p,el1,el2,ek1,ek2)
A

array([[5.2, 6.2, 7.2],
       [5.4, 6.4, 7.4],
       [5.6, 6.6, 7.6],
       [5.8, 6.8, 7.8]])

In [18]:
al1 = np.dot(A.T,el1); al2 = np.dot(A.T,el2)
ak1 = np.dot(A,ek1); ak2 = np.dot(A,ek2)

In [19]:
vz = update_vz(lambda_1, lambda_2, gamma_1, gamma_2, nu1, nu2, w_l, u_k, al1, al2, ak1, ak2)
v, z = vz[0], vz[1]
v

array([[-0. , -0.4, -0.2, -0. , -0.4, -0. ],
       [-0. , -0.4, -0.2, -0. , -0.4, -0. ],
       [-0. , -0.4, -0.2, -0. , -0.4, -0. ]])

In [20]:
z

array([[-1., -2., -1.],
       [-1., -2., -1.],
       [-1., -2., -1.],
       [-1., -2., -1.]])

In [21]:
lambdas = update_lambda(lambda_1,lambda_2,v, z, nu1,nu2, al1, al2, ak1, ak2)

In [22]:
lambda_1, lambda_2 = lambdas[0],lambdas[1]

In [23]:
lambda_1

array([[0.2, 0. , 0. , 0.6, 0. , 0.2],
       [0.2, 0. , 0. , 0.6, 0. , 0.2],
       [0.2, 0. , 0. , 0.6, 0. , 0.2]])

In [24]:
lambda_2

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])